# Bushfire Detection Model Development
Using the Edge Impulse Python SDK with TensorFlow and Keras to train model to detect bushfires vs. no bushfire within a given image.

#Install Dependencies

In [1]:
!python -m pip install tensorflow==2.12.0 edgeimpulse

# Import Packages

In [2]:
from tensorflow import keras
import edgeimpulse as ei
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import os

# Setup

In [3]:
# Settings
ei.API_KEY = "ei_c90f6349e4f2624274eff30ee840a3e6412c3135d1651ee15515404d290ce842"
labels = ["fire", "nofire"]
num_classes = len(labels)
deploy_filename = "bushfire_detection_model_cpp.zip"

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Prepare Dataset
Organise files between 'Training and Validation' and 'Testing' folders containing 'fire' and 'nofire' images.


In [5]:
# Define paths for your dataset
train_val_directory = 'drive/MyDrive/forest_fire/Training and Validation'
test_directory = 'drive/MyDrive/forest_fire/Testing'

# Create an ImageDataGenerator for training and validation
train_val_datagen = ImageDataGenerator(
    rescale=1.0 / 255,        # Normalize pixel values
    validation_split=0.2      # Use 20% of the training data for validation
)

# Create generators for training and validation data
train_generator = train_val_datagen.flow_from_directory(
    train_val_directory,
    target_size=(128, 128),   # Resize images to 128x128
    batch_size=32,             # Number of images to return in each batch
    class_mode='binary',       # Binary classification (fire or no fire)
    subset='training',          # Set as training data
    classes=['fire', 'nofire'] # Explicitly specify the class names
)

validation_generator = train_val_datagen.flow_from_directory(
    train_val_directory,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',       # Binary classification
    subset='validation',        # Set as validation data
    classes=['fire', 'nofire'] # Explicitly specify the class names
)

Found 1467 images belonging to 2 classes.
Found 365 images belonging to 2 classes.


Setup of 'Testing' directory

In [6]:
# Create an ImageDataGenerator for testing (no split needed)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Create a generator for the test data
test_generator = test_datagen.flow_from_directory(
    test_directory,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',       # Binary classification
    shuffle=False,             # Don't shuffle test data
    classes=['fire', 'nofire'] # Explicitly specify the class names
)

Found 68 images belonging to 2 classes.


#Building and compiling model

In [7]:
# Build your model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Testing model setup

In [8]:
print(train_generator.class_indices)

{'fire': 0, 'nofire': 1}


# Training Model

In [9]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

Epoch 1/10
45/45 [==============================] - 439s 10s/step - loss: 0.6099 - accuracy: 0.7073 - val_loss: 0.5384 - val_accuracy: 0.7983
Epoch 2/10
45/45 [==============================] - 28s 621ms/step - loss: 0.4021 - accuracy: 0.8669 - val_loss: 0.4175 - val_accuracy: 0.8210
Epoch 3/10
45/45 [==============================] - 27s 608ms/step - loss: 0.2874 - accuracy: 0.8878 - val_loss: 0.3784 - val_accuracy: 0.8438
Epoch 4/10
45/45 [==============================] - 26s 573ms/step - loss: 0.2533 - accuracy: 0.9052 - val_loss: 0.3330 - val_accuracy: 0.8778
Epoch 5/10
45/45 [==============================] - 28s 612ms/step - loss: 0.2237 - accuracy: 0.9108 - val_loss: 0.3174 - val_accuracy: 0.8864
Epoch 6/10
45/45 [==============================] - 28s 619ms/step - loss: 0.2003 - accuracy: 0.9233 - val_loss: 0.3022 - val_accuracy: 0.8807
Epoch 7/10
45/45 [==============================] - 28s 615ms/step - loss: 0.1919 - accuracy: 0.9261 - val_loss: 0.2860 - val_accuracy: 0.8977


#Test Accuracy and Evaluation

In [10]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f'Test Accuracy: {test_accuracy:.2f}')

2/2 [==============================] - 18s 18s/step - loss: 0.3056 - accuracy: 0.8906
Test Accuracy: 0.89


# Model to System Compatibility

In [11]:
# List the available profile target devices
# ei.model.list_profile_devices()

In [12]:
# Estimate the RAM, ROM, and inference time for our model on the target hardware family
# try:
#     profile = ei.model.profile(model=model,
#                                device='openmv-h7p')
#     print(profile.summary())
# except Exception as e:
#     print(f"Could not profile: {e}")

# Deploying and Downloading Model

In [13]:
# Define the output and input types for your model
model_output_type = ei.model.output_type.Classification(labels=labels)

# Define the model input type for images
model_input_type = ei.model.input_type.ImageInput()

# Define the filename for the deployment package
deploy_filename = "saved_model_zip.zip"

# Save your model as a TensorFlow SavedModel first
model.save("saved_model")  # Save model in the correct format

# Try to deploy the model directly as a C++ library in a .zip file
deploy_bytes = None
try:
    # Specify the path to the saved model
    deploy_bytes = ei.model.deploy(
        model="saved_model",  # Path to the saved model
        model_output_type=model_output_type,
        model_input_type=model_input_type,
        deploy_target='zip'  # Export as a .zip file with C++ library
    )
    print("Model deployment successful.")
except Exception as e:
    print(f"Could not deploy: {e}")

# Write the deployed model bytes to a file
if deploy_bytes:
    with open(deploy_filename, 'wb') as f:
        f.write(deploy_bytes.getvalue())
    print(f"Model exported as {deploy_filename}")
else:
    print("Deployment failed; no deployable bytes were generated.")

Model deployment successful.
Model exported as saved_model_zip.zip


In [14]:
from google.colab import files
import shutil

shutil.make_archive("saved_model", 'zip', "saved_model")
files.download("saved_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Model Specifications
Code to check if model has been trained correctly

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 global_average_pooling2d (G  (None, 32)               0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 64)                2112      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 3,073
Trainable params: 3,073
Non-trainabl